In [1]:
import pandas as pd
import numpy as np

In [2]:
gym_data = pd.read_csv("gym/gym_exercise_dataset.csv")
gym_data.head()

,Exercise Name,Equipment,Variation,Utility,Mechanics,Force,Preparation,Execution,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Main_muscle,Difficulty (1-5),Secondary Muscles,parent_id
0,Neck Flexion,Cable,No,Basic or Auxiliary,Isolated,Pull,Sit on bench facing away from middle pulley. P...,Move head away from pulley by bending neck for...,"Sternocleidomastoid,","None,","Rectus Abdominis, Obliques,",NaN,NaN,Neck,2,Sternocleidomastoid,NaN
1,Neck Flexion,Lever (plate loaded),No,Basic or Auxiliary,Isolated,Pull,Sit on seat in machine. Position padded lever ...,Move head forward by flexing neck until chin t...,"Sternocleidomastoid,","None,","Latissimus Dorsi, Deltoid, Posterior, Rhomboid...",NaN,NaN,Neck,2,Sternocleidomastoid,NaN
2,Lateral Neck Flexion,Lever (plate loaded),No,Auxiliary,Isolated,Pull,Sit on seat in machine with feet apart . Pos...,Move head down to side by laterally flexing ne...,"Sternocleidomastoid,","Splenius, Erector Spinae, Levator Scapulae, Tr...","Latissimus Dorsi, Pectoralis Major, Sternal, P...",NaN,NaN,Neck,2,"Sternocleidomastoid, Levator Scapulae",NaN
3,Neck Flexion,Lever (selectorized),No,Basic or Auxiliary,Isolated,Pull,Sit on seat in machine. Position padded lever ...,Move head forward by flexing neck until chin t...,"Sternocleidomastoid,","None,","Latissimus Dorsi, Deltoid, Posterior, Rhomboid...",NaN,NaN,Neck,2,Sternocleidomastoid,NaN
4,Lateral Neck Flexion,Lever (selectorized),No,Auxiliary,Isolated,Pull,Sit on seat in machine with feet apart. Positi...,Move head down to side by laterally flexing ne...,"Sternocleidomastoid,","Splenius, Erector Spinae, Levator Scapulae, Tr...","Latissimus Dorsi, Pectoralis Major, Sternal, P...",NaN,NaN,Neck,2,"Sternocleidomastoid, Levator Scapulae",NaN


In [3]:
exe_data = pd.read_csv("gym/Exersices.csv")
exe_data.head()

,Unnamed: 0,bodyPart,equipment,name,target,instructions
0,0,back,cable,alternate lateral pulldown,lats,['Sit on the cable machine with your back stra...
1,1,back,body weight,archer pull up,lats,['Start by hanging from a pull-up bar with an ...
2,2,back,leverage machine,assisted parallel close grip pull-up,lats,['Adjust the machine to your desired weight an...
3,3,back,leverage machine,assisted pull-up,lats,['Adjust the machine to your desired weight an...
4,4,back,leverage machine,assisted standing chin-up,lats,['Adjust the machine to your desired assistanc...


In [4]:

# exe_data.columns

In [5]:
df= exe_data['target'].unique()
df

array(['lats', 'spine', 'upper back', 'traps', 'cardiovascular system',
       'pectorals', 'serratus anterior', 'forearms', 'calves',
       'levator scapulae', 'delts', 'triceps', 'biceps', 'quads',
       'glutes', 'hamstrings', 'adductors', 'abductors', 'abs'],
      dtype=object)

In [6]:
# exe_data.dtypes

In [7]:
# gym_data.columns

In [8]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score

# # Load the dataset
# # Assuming the dataset is a CSV
# df = pd.read_csv('gym/Exercises.csv')

# # Label encoding for categorical columns
# label_encoders = {}
# categorical_columns = ['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1',
#                        'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']

# # Encode each categorical column
# for col in categorical_columns:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
#     label_encoders[col] = le  # Save the encoders for later use

# # Selecting features
# X = df[['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1', 
#         'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']]

# # Target variable (target muscle)
# y = df['target']

# # Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Random Forest model
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Train the model
# rf.fit(X_train, y_train)

# # Predict on test set
# y_pred_rf = rf.predict(X_test)

# # Evaluate accuracy
# accuracy_rf = accuracy_score(y_test, y_pred_rf)
# print(f'Random Forest Accuracy: {accuracy_rf * 100:.2f}%')
# # Example: Recommend exercises similar to a specific target
# def recommend_exercises(target_input, k=5):
#     # Encode the target input using the label encoder
#     target_encoded = label_encoders['target'].transform([target_input])[0]
    
#     # Get the predicted probabilities for the input
#     probabilities = rf.predict_proba(X_test)
    
#     # # Get indices of top k exercises similar to the input
#     recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
#     # # Decode the recommended target exercises
#     recommendations = df.iloc[recommended_indices]
    
#     # Return the recommended exercises
#     return recommendations

# # Example usage
# target_input = 'triceps'  # Provide a target muscle group
# recommended_exercises = recommend_exercises(target_input)
# print(recommended_exercises)





In [9]:
# # KNN model
# knn = KNeighborsClassifier(n_neighbors=5)

# # Train the model
# knn.fit(X_train, y_train)

# # Predict on test set
# y_pred_knn = knn.predict(X_test)

# # Evaluate accuracy
# accuracy_knn = accuracy_score(y_test, y_pred_knn)
# print(f'KNN Accuracy: {accuracy_knn * 100:.2f}%')



In [10]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# import joblib  # For saving and loading the model
# import os

# # Load the dataset (update the path as needed)
# df = pd.read_csv('gym/exercises.csv')

# # Label encoding for categorical columns
# label_encoders = {}
# categorical_columns = ['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1',
#                        'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']

# # Encode each categorical column
# for col in categorical_columns:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
#     label_encoders[col] = le  # Save the encoders for later use

# # Selecting features
# X = df[['bodyPart', 'equipment', 'target', 'secondaryMuscles/0', 'secondaryMuscles/1', 
#         'secondaryMuscles/2', 'secondaryMuscles/3', 'secondaryMuscles/4', 'secondaryMuscles/5']]

# # Target variable (target muscle)
# y = df['target']

# # Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Random Forest model
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Train the model
# rf.fit(X_train, y_train)

# # Predict on test set
# y_pred_rf = rf.predict(X_test)

# # Evaluate accuracy
# accuracy_rf = accuracy_score(y_test, y_pred_rf)
# print(f'Random Forest Accuracy: {accuracy_rf * 100:.2f}%')

# # Save the trained model as a .pkl file
# model_filename = 'exercise_recommender_model.pkl'
# joblib.dump(rf, model_filename)
# print(f'Model saved as {model_filename}')

# # Save the label encoders for future use
# encoder_filename = 'label_encoders.pkl'
# joblib.dump(label_encoders, encoder_filename)
# print(f'Label encoders saved as {encoder_filename}')

# # Function to recommend exercises similar to a specific target
# def recommend_exercises(target_input, k=5):
#     # Error handling for invalid inputs
#     if target_input not in label_encoders['target'].classes_:
#         return f"Invalid target: {target_input}. Available options: {label_encoders['target'].classes_}"
    
#     # Encode the target input using the label encoder
#     target_encoded = label_encoders['target'].transform([target_input])[0]
    
#     # Get the predicted probabilities for all test data
#     probabilities = rf.predict_proba(X_test)
    
#     # Get indices of top k exercises similar to the input
#     recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
#     # Decode the recommended target exercises
#     recommendations = df.iloc[recommended_indices]
    
#     # Return the recommended exercises
#     return recommendations

# # Example usage
# target_input = 'quads'  # Provide a target muscle group
# recommended_exercises = recommend_exercises(target_input)
# print(recommended_exercises)


In [11]:
# import joblib
# # # Save the RandomForest model and label encoders
# joblib.dump(rf, 'exe_model.pkl')
# joblib.dump(label_encoders, 'exe_label_encoders.pkl')

In [12]:
# # Example: Recommend exercises similar to a specific target
# def recommend_exercises(target_input, k=5):
#     target_encoded = label_encoders['target'].transform([target_input])[0]
    
#     # Find the k-nearest neighbors
#     distances, indices = knn.kneighbors(X[X['target'] == target_encoded], n_neighbors=k)
    
#     # Get the recommended exercises
#     recommended_exercises = df.iloc[indices[0]]
#     return recommended_exercises[['name', 'target', 'bodyPart', 'equipment']]

# # Recommend exercises similar to 'biceps'
# recommend_exercises('biceps', k=5)


In [13]:
# # Feature importance from the random forest
# importances = rf.feature_importances_
# feature_names = X.columns

# # Display the feature importance
# for name, importance in zip(feature_names, importances):
#     print(f'{name}: {importance:.4f}')


In [14]:
# import pickle
# # Save the Random Forest model and label encoders
# with open('exercise_model.pkl', 'wb') as file:
#     pickle.dump({'model': rf, 'label_encoders': label_encoders}, file)

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
# Assuming the dataset is a CSV
df = pd.read_csv('./gym/Exersices.csv')

# Label encoding for categorical columns
label_encoders = {}
categorical_columns = ['bodyPart', 'equipment', 'target']

# Encode each categorical column
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
    label_encoders[col] = le  # Save the encoders for later use

# Selecting features
X = df[['bodyPart', 'equipment', 'target']]

# Target variable (target muscle)
y = df['target']

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf.predict(X_test)

# Evaluate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Accuracy: {accuracy_rf * 100:.2f}%')
# Example: Recommend exercises similar to a specific target
def recommend_exercises(target_input, k=5):
    # Encode the target input using the label encoder
    target_encoded = label_encoders['target'].transform([target_input])[0]
    
    # Get the predicted probabilities for the input
    probabilities = rf.predict_proba(X_test)
    
    # # Get indices of top k exercises similar to the input
    recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
    # # Decode the recommended target exercises
    recommendations = df.iloc[recommended_indices]
    
    # Return the recommended exercises
    return recommendations

# Example usage
target_input = 'traps'  # Provide a target muscle group
recommended_exercises = recommend_exercises(target_input)
print(recommended_exercises)





Random Forest Accuracy: 99.25%
     Unnamed: 0  bodyPart  equipment                            name  target  \
152         152         0         16                   london bridge      18   
169         169         0         15             roller back stretch      15   
156         156         0          3              mixed grip chin-up      10   
153         153         0          3                 lower back curl      15   
62           62         0          5  cable rope elevated seated row      18   

                                          instructions  
152  ['Attach the rope to a high anchor point.', 'S...  
169  ['Start by sitting on the ground with your leg...  
156  ['Grab the pull-up bar with an underhand grip ...  
153  ['Lie flat on your stomach with your legs exte...  
62   ['Sit on the elevated seat facing the cable ma...  


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv('./gym/Exersices.csv')

# Label encoding for categorical columns
label_encoders = {}
categorical_columns = ['bodyPart', 'equipment', 'target']

# Encode each categorical column
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
    label_encoders[col] = le  # Save the encoders for later use

# Selecting features
X = df[['bodyPart', 'equipment', 'target']]
y = df['target']  # Target variable

# Standardize features for KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the KNN model
k = 5
knn = KNeighborsClassifier(n_neighbors=k)

# Train the model
knn.fit(X_train, y_train)

# Predict on test set to evaluate accuracy
y_pred_knn = knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f'KNN Accuracy: {accuracy_knn * 100:.2f}%')

# Recommendation function using KNN
def recommend_exercises(target_input, k=5):
    # Encode the target input using the label encoder
    target_encoded = label_encoders['target'].transform([target_input])[0]
    
    # Create a sample input with the encoded target value
    sample_input = X[X['target'] == target_encoded].iloc[0]  # Get one example with this target
    
    # Standardize the sample input for KNN compatibility
    sample_input_scaled = scaler.transform([sample_input])
    
    # Find the k nearest neighbors
    distances, indices = knn.kneighbors(sample_input_scaled, n_neighbors=k)
    
    # Decode the recommended target exercises
    recommendations = df.iloc[indices[0]]
    
    # Return the recommended exercises
    return recommendations

# Example usage
target_input = 'traps'  # Provide a target muscle group
recommended_exercises = recommend_exercises(target_input)
print(recommended_exercises)


KNN Accuracy: 94.34%
     Unnamed: 0  bodyPart  equipment  \
798         798         7          6   
166         166         0         11   
781         781         7          6   
456         456         4          6   
491         491         5          3   

                                                  name  target  \
798                dumbbell one arm seated hammer curl       3   
166                  reverse grip machine lat pulldown      10   
781  dumbbell lying one arm supinated triceps exten...      17   
456                     dumbbell single leg calf raise       4   
491                                  neck side stretch      11   

                                          instructions  
798  ['Sit on a bench with your back straight and f...  
166  ['Adjust the seat height and position yourself...  
781  ['Lie flat on a bench with your back and head ...  
456  ['Stand on the edge of a step or platform with...  
491  ['Stand or sit up straight with your shoulders...  

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [17]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split  # for data splitting

# Load the dataset
df = pd.read_csv('./gym/Exersices.csv')

# Label encoding for categorical columns
categorical_columns = ['bodyPart', 'equipment', 'target']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))  # Handle NaN values if necessary
    label_encoders[col] = le

# Selecting features and target
X = df[['bodyPart', 'equipment', 'target']].values
y = df['target'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree Node
class DecisionNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature      # index of the feature to split on
        self.threshold = threshold  # threshold value for the split
        self.left = left            # left subtree
        self.right = right          # right subtree
        self.value = value          # leaf node value (if any)

# Calculate Gini impurity
def gini(y):
    counts = np.bincount(y)
    probabilities = counts / len(y)
    return 1 - np.sum(probabilities ** 2)

# Split dataset
def split_dataset(X, y, feature, threshold):
    left_indices = np.where(X[:, feature] <= threshold)[0]
    right_indices = np.where(X[:, feature] > threshold)[0]
    return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

# Grow tree
def grow_tree(X, y, depth=0, max_depth=10):
    num_samples, num_features = X.shape
    if num_samples <= 1 or depth >= max_depth:
        leaf_value = most_common_label(y)
        return DecisionNode(value=leaf_value)

    best_gini = 1.0
    best_feature, best_threshold = None, None

    for feature in range(num_features):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            X_left, X_right, y_left, y_right = split_dataset(X, y, feature, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            gini_left = gini(y_left)
            gini_right = gini(y_right)
            weighted_gini = (len(y_left) * gini_left + len(y_right) * gini_right) / len(y)
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_feature = feature
                best_threshold = threshold

    if best_gini == 1.0:
        leaf_value = most_common_label(y)
        return DecisionNode(value=leaf_value)

    X_left, X_right, y_left, y_right = split_dataset(X, y, best_feature, best_threshold)
    left_child = grow_tree(X_left, y_left, depth + 1, max_depth)
    right_child = grow_tree(X_right, y_right, depth + 1, max_depth)
    return DecisionNode(best_feature, best_threshold, left_child, right_child)

# Majority voting for Random Forest predictions
def most_common_label(y):
    counter = Counter(y)
    return counter.most_common(1)[0][0]

# Decision Tree Classifier
class DecisionTreeClassifier:
    def __init__(self, max_depth=10):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = grow_tree(X, y, max_depth=self.max_depth)

    def _predict(self, inputs):
        node = self.root
        while node.value is None:
            if inputs[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

# Random Forest Classifier
class RandomForestClassifierFromScratch:
    def __init__(self, n_trees=10, max_depth=10, sample_size=None, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.sample_size = sample_size
        self.n_features = n_features
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, self.sample_size or n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeClassifier(max_depth=self.max_depth)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

# Training the Random Forest from scratch
rf_scratch = RandomForestClassifierFromScratch(n_trees=10, max_depth=10)
rf_scratch.fit(X_train, y_train)
y_pred_scratch = rf_scratch.predict(X_test)

# Evaluate accuracy
accuracy_scratch = np.mean(y_pred_scratch == y_test)
print(f'Random Forest from Scratch Accuracy: {accuracy_scratch * 100:.2f}%')


Random Forest from Scratch Accuracy: 100.00%


In [23]:
import matplotlib.pyplot as plt

# Plotting the accuracy of the Random Forest model
accuracy_scratch = np.mean(y_pred_scratch == y_test)
plt.bar(['Random Forest from Scratch'], [accuracy_scratch * 100])
plt.ylabel('Accuracy (%)')
plt.title('Random Forest Accuracy')
plt.show()

ValueError: operands could not be broadcast together with shapes (265,) (264,) 

In [22]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Visualize the first decision tree in the Random Forest
tree = rf_scratch.trees[0]
plt.figure(figsize=(12,8))
plot_tree(tree.root, filled=True, feature_names=['bodyPart', 'equipment', 'target'])
plt.show()

InvalidParameterError: The 'decision_tree' parameter of plot_tree must be an instance of 'sklearn.tree._classes.DecisionTreeClassifier' or an instance of 'sklearn.tree._classes.DecisionTreeRegressor'. Got <__main__.DecisionNode object at 0x000001210ADA6610> instead.

<Figure size 1200x800 with 0 Axes>

In [18]:
# import pandas as pd
# import numpy as np
# from collections import Counter
# from sklearn.model_selection import train_test_split

# # Load and preprocess dataset
# df = pd.read_csv('./gym/Exersices.csv')
# categorical_columns = ['bodyPart', 'equipment', 'target']
# label_encoders = {}
# for col in categorical_columns:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col].astype(str))
#     label_encoders[col] = le

# X = df[['bodyPart', 'equipment', 'target']].values
# y = df['target'].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# class DecisionNode:
#     def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
#         self.feature = feature
#         self.threshold = threshold
#         self.left = left
#         self.right = right
#         self.value = value

# def gini(y):
#     counts = np.bincount(y)
#     probabilities = counts / len(y)
#     return 1 - np.sum(probabilities ** 2)

# def split_dataset(X, y, feature, threshold):
#     left_indices = np.where(X[:, feature] <= threshold)[0]
#     right_indices = np.where(X[:, feature] > threshold)[0]
#     return X[left_indices], X[right_indices], y[left_indices], y[right_indices]

# def grow_tree(X, y, depth=0, max_depth=10):
#     num_samples, num_features = X.shape
#     if num_samples <= 1 or depth >= max_depth:
#         leaf_value = most_common_label(y)
#         return DecisionNode(value=leaf_value)

#     best_gini = 1.0
#     best_feature, best_threshold = None, None

#     for feature in range(num_features):
#         thresholds = np.unique(X[:, feature])
#         for threshold in thresholds:
#             X_left, X_right, y_left, y_right = split_dataset(X, y, feature, threshold)
#             if len(y_left) == 0 or len(y_right) == 0:
#                 continue
#             gini_left = gini(y_left)
#             gini_right = gini(y_right)
#             weighted_gini = (len(y_left) * gini_left + len(y_right) * gini_right) / len(y)
#             if weighted_gini < best_gini:
#                 best_gini = weighted_gini
#                 best_feature = feature
#                 best_threshold = threshold

#     if best_gini == 1.0:
#         leaf_value = most_common_label(y)
#         return DecisionNode(value=leaf_value)

#     X_left, X_right, y_left, y_right = split_dataset(X, y, best_feature, best_threshold)
#     left_child = grow_tree(X_left, y_left, depth + 1, max_depth)
#     right_child = grow_tree(X_right, y_right, depth + 1, max_depth)
#     return DecisionNode(best_feature, best_threshold, left_child, right_child)

# def most_common_label(y):
#     counter = Counter(y)
#     return counter.most_common(1)[0][0]

# class DecisionTreeClassifier:
#     def __init__(self, max_depth=10):
#         self.max_depth = max_depth
#         self.root = None

#     def fit(self, X, y):
#         self.root = grow_tree(X, y, max_depth=self.max_depth)

#     def _predict(self, inputs):
#         node = self.root
#         while node.value is None:
#             if inputs[node.feature] <= node.threshold:
#                 node = node.left
#             else:
#                 node = node.right
#         return node.value

#     def predict(self, X):
#         return np.array([self._predict(inputs) for inputs in X])

# class RandomForestClassifierFromScratch:
#     def __init__(self, n_trees=10, max_depth=10, sample_size=None, n_features=None):
#         self.n_trees = n_trees
#         self.max_depth = max_depth
#         self.sample_size = sample_size
#         self.n_features = n_features
#         self.trees = []

#     def _bootstrap_sample(self, X, y):
#         n_samples = X.shape[0]
#         indices = np.random.choice(n_samples, self.sample_size or n_samples, replace=True)
#         return X[indices], y[indices]

#     def fit(self, X, y):
#         self.trees = []
#         for _ in range(self.n_trees):
#             tree = DecisionTreeClassifier(max_depth=self.max_depth)
#             X_sample, y_sample = self._bootstrap_sample(X, y)
#             tree.fit(X_sample, y_sample)
#             self.trees.append(tree)

#     def predict(self, X):
#         tree_preds = np.array([tree.predict(X) for tree in self.trees])
#         tree_preds = np.swapaxes(tree_preds, 0, 1)
#         y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
#         return np.array(y_pred)

#     def predict_proba(self, X):
#         # Aggregates the probabilities from all trees
#         tree_preds = np.array([tree.predict(X) for tree in self.trees])
#         probas = []
#         for i in range(X.shape[0]):
#             class_counts = Counter(tree_preds[:, i])
#             probas.append([class_counts.get(c, 0) / self.n_trees for c in range(len(np.unique(y)))])
#         return np.array(probas)

# # Initialize and train the model
# rf_scratch = RandomForestClassifierFromScratch(n_trees=10, max_depth=10)
# rf_scratch.fit(X_train, y_train)


# # Predict on test set
# y_pred = rf_scratch.predict(X_test)

# # Calculate accuracy
# accuracy = np.mean(y_pred == y_test)
# print(f'Random Forest Accuracy (from scratch): {accuracy * 100:.2f}%')


# # Exercise recommendation based on target muscle group
# def recommend_exercises(target_input, k=5):
#     # Encode the target input using the label encoder
#     target_encoded = label_encoders['target'].transform([target_input])[0]
    
#     # Get the predicted probabilities for the test set
#     probabilities = rf_scratch.predict_proba(X_test)
    
#     # Find top k exercises with the highest probability for the target class
#     recommended_indices = np.argsort(probabilities[:, target_encoded])[-k:][::-1]
    
#     # Decode the recommended target exercises
#     recommendations = df.iloc[recommended_indices]
    
#     # Return the recommended exercises
#     return recommendations

# # Example usage
# target_input = 'traps'  # Example muscle group to target
# recommended_exercises = recommend_exercises(target_input)
# print(recommended_exercises)


In [19]:
import pandas as pd
import numpy as np
from collections import Counter

# Load the dataset
df = pd.read_csv('./gym/Exersices.csv')

# Manually encode categorical columns
def manual_label_encode(column):
    unique_values = column.unique()
    encoding_map = {value: idx for idx, value in enumerate(unique_values)}
    encoded_column = column.map(encoding_map)
    return encoded_column, encoding_map

# Encode categorical columns
categorical_columns = ['bodyPart', 'equipment', 'target']
encoding_maps = {}

for col in categorical_columns:
    df[col], encoding_maps[col] = manual_label_encode(df[col])

# Split features and target
X = df[['bodyPart', 'equipment', 'target']].values
y = df['target'].values

# Manually standardize features
def standardize(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    X_standardized = (X - mean) / std
    return X_standardized, mean, std

X_standardized, mean, std = standardize(X)

# Split data into training and test sets
def train_test_split_custom(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    test_set_size = int(len(X) * test_size)
    test_indices = indices[:test_set_size]
    train_indices = indices[test_set_size:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

X_train, X_test, y_train, y_test = train_test_split_custom(X_standardized, y, test_size=0.2)

# Implement K-Nearest Neighbors from scratch
class KNNClassifier:
    def __init__(self, k=5):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict_single(x) for x in X]
        return np.array(predictions)

    def _predict_single(self, x):
        # Calculate distances from the point x to all training samples
        distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))  # Euclidean distance calculation
        # Get the indices of the k nearest neighbors
        nearest_neighbor_ids = distances.argsort()[:self.k]
        # Extract the labels of the nearest neighbors
        nearest_neighbor_labels = self.y_train[nearest_neighbor_ids]
        # Get the most common label among the neighbors
        most_common = Counter(nearest_neighbor_labels).most_common(1)
        return most_common[0][0]

# Initialize and train the KNN model
k = 5
knn = KNNClassifier(k=k)
knn.fit(X_train, y_train)

# Predict on test set to evaluate accuracy
y_pred = knn.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f'KNN Accuracy: {accuracy * 100:.2f}%')

# # Recommendation function using the KNN model
# def recommend_exercises(target_input, k=5):
#     # Encode the target input
#     target_encoded = encoding_maps['target'][target_input]
    
#     # Create a sample input with the encoded target value
#     sample_input = df[df['target'] == target_encoded].iloc[0][['bodyPart', 'equipment', 'target']].values
    
#     # Standardize the sample input (ensure it is 2D for distance calculation)
#     sample_input_standardized = (sample_input - mean) / std
#     sample_input_standardized = sample_input_standardized.reshape(1, -1)  # Reshape to 2D array

#     # Find k nearest neighbors
#     distances = np.sqrt(np.sum((X_standardized - sample_input_standardized) ** 2, axis=1))
#     nearest_neighbor_ids = distances.argsort()[:k]
    
#     # Retrieve recommended exercises
#     recommendations = df.iloc[nearest_neighbor_ids]
#     return recommendations

# Example usage
# target_input = 'traps'  # Provide a target muscle group
# recommended_exercises = recommend_exercises(target_input)
# print(recommended_exercises)


KNN Accuracy: 93.94%
